# Convert a csv database into posgres

This follows the notebook: ../general-docs/python_sql_dev_setups.ipynb


In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [2]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'autism-docs'
username = 'rangel'

In [3]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print (engine.url)

postgres://rangel@localhost/autism-docs


In [4]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [5]:
# read a database from CSV and load it into a pandas dataframe
df = pd.DataFrame.from_csv('articles-n-forums-posts.csv')
df.head(2)

,category,href,source,text,title,tokens,text_short
0,['category-autism-therapy'],https://www.autismparentingmagazine.com/autism...,https://www.autismparentingmagazine.com/,For children with autism spectrum disorder (AS...,"Autism, Head Banging and other Self Harming Be...","['autism', 'head', 'banging', 'and', 'other', ...",For children with autism spectrum disorder (AS...
1,['category-autism-therapy'],https://www.autismparentingmagazine.com/high-q...,https://www.autismparentingmagazine.com/,Dr. Stephen Shore once said “If you’ve met one...,High Quality ABA Treatment: What Every Parent...,"['high', 'quality', 'aba', 'treatment', 'what'...",Dr. Stephen Shore once said “If you’ve met one...


In [6]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
df.to_sql('articles-n-forums-posts', engine, if_exists='replace')

In [7]:
## Now try the same queries, but in python!

# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM \"articles-n-forums-posts\" WHERE title LIKE '%autism%';
"""
query = pd.read_sql_query(sql_query,con)

query.head()

,index,category,href,source,text,title,tokens,text_short
0,235,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,\n Thank You I had no clue what was wro...,I am diagnosed with autism but id it true...?,"['i', 'am', 'diagnosed', 'with', 'autism', 'bu...",\n Thank You I had no clue what was wro...
1,21,['category-autism-and-diet'],https://www.autismparentingmagazine.com/sugary...,https://www.autismparentingmagazine.com/,Sivakumari asks: My daughter loves to eat ice-...,Q and A Do sugary foods make children with aut...,"['q', 'and', 'a', 'do', 'sugary', 'foods', 'ma...",Sivakumari asks: My daughter loves to eat ice-...
2,257,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,\n Forgot the link. \nhttp://ezineartic...,New treatment for autism?,"['new', 'treatment', 'for', 'autism', 'forgot'...",\n Forgot the link. \nhttp://ezineartic...
3,233,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,\n Hi! My little boy was diagnosed with...,is my 2years son normal or he have autism,"['is', 'my', '2years', 'son', 'normal', 'or', ...",\n Hi! My little boy was diagnosed with...
4,231,['forums'],http://www.medhelp.org//posts/Autism--Asperger...,http://www.medhelp.org,\n My EEG was classified as normal with...,does the e.e.g confirm if the baby has autism,"['does', 'the', 'e', 'e', 'g', 'confirm', 'if'...",\n My EEG was classified as normal with...
